In [1]:
import os
import pandas as pd
import numpy as np
from stackregression import stack_regression, stack_prediction, get_prediction, print_prediction_report
from utils import encode_numeric_zscore_list, encode_numeric_zscore_all, to_xy
from sklearn import preprocessing
from sklearn.cross_validation import train_test_split

/home/andy/anaconda3/envs/tensorflow/lib/python3.5/site-packages/sklearn/cross_validation.py:44: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)
Using Theano backend.


In [2]:
#Read Input CSV file
path = "./data/self"
inputFilePath = os.path.join(path, "TestRegression.csv")
#df = pd.read_csv(inputFilePath, compression="zip", header=0, na_values=['NULL'])
df = pd.read_csv(inputFilePath, header=0, na_values=['NULL'])

In [3]:
#shuffle dataset. Unnecessary in this case because already sorted by guid
np.random.seed(42)
df = df.reindex(np.random.permutation(df.index))

#df.drop('id', axis=1, inplace=True)
#encode categoricals as dummies

#encode_text_dummy_list(df, ['cat1', 'cat2', 'cat3', 'cat4', 'cat5', 'cat6', 'cat7', 'cat8', 'cat9', 'cat10', 'cat11', 'cat12', 'cat13', 'cat14', 'cat15', 'cat16', 'cat17', 'cat18', 'cat19', 'cat20', 'cat21', 'cat22', 'cat23', 'cat24', 'cat25', 'cat26', 'cat27', 'cat28', 'cat29', 'cat30', 'cat31', 'cat32', 'cat33', 'cat34', 'cat35', 'cat36', 'cat37', 'cat38', 'cat39', 'cat40', 'cat41', 'cat42', 'cat43', 'cat44', 'cat45', 'cat46', 'cat47', 'cat48', 'cat49', 'cat50', 'cat51', 'cat52', 'cat53', 'cat54', 'cat55', 'cat56', 'cat57', 'cat58', 'cat59', 'cat60', 'cat61', 'cat62', 'cat63', 'cat64', 'cat65', 'cat66', 'cat67', 'cat68', 'cat69', 'cat70', 'cat71', 'cat72', 'cat73', 'cat74', 'cat75', 'cat76', 'cat77', 'cat78', 'cat79', 'cat80', 'cat81', 'cat82', 'cat83', 'cat84', 'cat85', 'cat86', 'cat87', 'cat88', 'cat89', 'cat90', 'cat91', 'cat92', 'cat93', 'cat94', 'cat95', 'cat96', 'cat97', 'cat98', 'cat99', 'cat100', 'cat101', 'cat102', 'cat103', 'cat104', 'cat105', 'cat106', 'cat107', 'cat108', 'cat109', 'cat110', 'cat111', 'cat112', 'cat113', 'cat114', 'cat115', 'cat116'])

#encode all numeric values to zscored values
#encode_numeric_zscore(df, 'AW')

#encode_numeric_log_list(df, ['A', 'B', 'C', 'D'])
encode_numeric_zscore_list(df, ['A', 'B', 'C', 'D'])

#discard rows where z-score > 2
df.fillna(0)

# Create x(predictors) and y (expected outcome)
x,y = to_xy(df, "Label")
x = preprocessing.StandardScaler().fit_transform(x)
x = x.astype(np.float32)
y = y.astype(np.float32)

headers = list(df.columns.values)
print(headers)
#print(df.columns.values)

    
step1predictions, learners = stack_regression(x, y)
print("Step 1 Predictions:")
print("---------------------------------------------")
print_prediction_report(step1predictions, y)
print("---------------------------------------------")

encode_numeric_zscore_all(step1predictions)
x = np.concatenate((x, step1predictions), axis=1)
x = x.astype(np.float32)
step2predictions, step2_learners = stack_regression(x, y)

print("Step 2 Predictions:")
print("---------------------------------------------")
print_prediction_report(step2predictions, y)
print("---------------------------------------------")

#encode_numeric_zscore_all(step1predictions_ytest)
#x_test = np.concatenate((x_test, step1predictions_ytest), axis=1)
#x_test = x_test.astype(np.float32)
#step2predictions_ytest = stack_prediction(x_test, y_test, step2_learners)



float64
['A', 'B', 'C', 'D', 'Label']
Epoch 1/10
666/666 [==============================] - 0s - loss: 360.4357     
Epoch 2/10
666/666 [==============================] - 1s - loss: 324.0519     
Epoch 3/10
666/666 [==============================] - 0s - loss: 224.8281     
Epoch 4/10
666/666 [==============================] - 0s - loss: 118.6471     
Epoch 5/10
666/666 [==============================] - 1s - loss: 71.3714     
Epoch 6/10
666/666 [==============================] - 0s - loss: 57.5920     
Epoch 7/10
666/666 [==============================] - 0s - loss: 51.8882     
Epoch 8/10
666/666 [==============================] - 0s - loss: 52.9071     
Epoch 9/10
666/666 [==============================] - 0s - loss: 51.7914     
Epoch 10/10
666/666 [==============================] - 1s - loss: 46.9312     
Epoch 1/10
667/667 [==============================] - 1s - loss: 347.5959     
Epoch 2/10
667/667 [==============================] - 1s - loss: 313.9388     
Epoch 3/10
667/667 